In [5]:
import pandas as pd
import numpy as np
import pickle

import seaborn as sns
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (10,8)

from itertools import combinations
import researchpy as rp

from sklearn.preprocessing import RobustScaler

from sklearn.preprocessing import OneHotEncoder 

from imblearn.over_sampling import RandomOverSampler

from imblearn.combine import SMOTETomek
from sklearn.model_selection import train_test_split


import warnings
warnings.filterwarnings('ignore')

from IPython.core.interactiveshell import InteractiveShell 
InteractiveShell.ast_node_interactivity = "all"

pd.options.display.max_columns = None 

In [6]:
df = pd.read_pickle('../data/cardio_risk.pkl')
df.head()

,education,sex,age_category,BMI_category,is_smoking,BPMeds,prevalentStroke,prevalentHyp,diabetes,age,cigsPerDay,totChol,sysBP,diaBP,BMI,heartRate,glucose,TenYearCHD
0,2.0,F,Senior Citizens,Overweight,YES,0.0,0,0,0,64.0,3.0,221.0,148.0,85.0,27.62,90.0,80.0,1
1,4.0,M,Middle-Aged Adults,Overweight,NO,0.0,0,1,0,36.0,0.0,212.0,168.0,98.0,29.77,72.0,75.0,0
2,1.0,F,Middle-Aged Adults,Normal Weight,YES,0.0,0,0,0,46.0,10.0,250.0,116.0,71.0,20.35,88.0,94.0,0
3,1.0,M,Middle-Aged Adults,Overweight,YES,0.0,0,1,0,50.0,20.0,233.0,158.0,88.0,28.26,68.0,94.0,1
4,1.0,F,Senior Citizens,Overweight,YES,0.0,0,0,0,64.0,30.0,241.0,136.5,85.0,26.42,70.0,77.0,0


In [7]:
df.drop(['diaBP'], axis = 1, inplace = True)

In [8]:
patient_1 = {'education': 4.0, 'sex': 'F', 'age_category': 'Senior Citizens', 'BMI_category': 'Overweight', 'is_smoking': 'YES', 'BPMeds': 1.0, 'prevalentStroke': 1,
        'prevalentHyp': 1, 'diabetes': 1, 'age': 63, 'cigsPerDay': 30.0, 'totChol': 242.0, 'sysBP': 138.0, 'BMI': 29.75, 'heartRate': 99.0, 'glucose': 182.0}

In [9]:
patient_1_df = pd.DataFrame (patient_1, index = [0])
patient_1_df 

,education,sex,age_category,BMI_category,is_smoking,BPMeds,prevalentStroke,prevalentHyp,diabetes,age,cigsPerDay,totChol,sysBP,BMI,heartRate,glucose
0,4.0,F,Senior Citizens,Overweight,YES,1.0,1,1,1,63,30.0,242.0,138.0,29.75,99.0,182.0


In [12]:
with open ('../data/cardio_processed_over.pkl', 'wb') as f:
    pickle.dump(robust, f)

NameError: name 'robust' is not defined

In [2]:
with open('../data/cardio_processed_over.pkl', "rb") as scaler:
    scaler = pickle.load(scaler)

In [13]:
scaler

,age_category,BMI_category,BPMeds,prevalentStroke,prevalentHyp,diabetes,age,cigsPerDay,totChol,sysBP,BMI,heartRate,glucose,education_1.0,education_2.0,education_3.0,education_4.0,is_smoking_NO,is_smoking_YES,sex_F,sex_M,TenYearCHD
0,2.0,0.25,0,0,0,0,1.071429,0.15,-0.224138,0.722222,0.450931,1.000000,0.142857,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,1
1,0.5,0.25,0,0,1,0,-0.928571,0.00,-0.379310,1.462963,0.883744,-0.200000,-0.214286,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0
2,0.5,0.00,0,0,0,0,-0.214286,0.50,0.275862,-0.462963,-1.012582,0.866667,1.142857,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0
3,0.5,0.25,0,0,1,0,0.071429,1.00,-0.017241,1.092593,0.579768,-0.466667,1.142857,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1
4,2.0,0.25,0,0,0,0,1.071429,1.50,0.120690,0.296296,0.209361,-0.333333,-0.071429,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5753,2.0,0.00,0,3,1,0,0.785714,0.75,0.344828,1.796296,-0.424761,1.133333,0.071429,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1
5754,0.5,0.25,0,0,1,0,-0.071429,0.00,0.706897,1.796296,-0.032209,0.000000,0.285714,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1
5755,0.5,0.50,0,0,1,0,0.428571,0.00,0.086207,1.129630,1.403120,0.000000,-0.500000,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1
5756,0.5,0.25,2,0,1,0,0.571429,0.50,0.655172,1.055556,-0.046301,-0.333333,1.214286,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,1


In [14]:
scaler.fit_transform(patient_1_df )

AttributeError: 'DataFrame' object has no attribute 'fit_transform'